In [1]:

using DataFrames
using RDatasets
using Statistics
using StatsPlots
using CSV
using StatsBase
using NamedArrays
using PrettyTables
using LIBSVM
using Printf
using SVR
using DecisionTree
using Shuffle
using Plots
using Distributions
using Zygote
using Flux

In [3]:
df = CSV.read(joinpath(dirname(@__FILE__),"Concrete_Data.csv"), DataFrame)

Row,Cement (kg/m3),Blast Furnace Slag (kg/m3),Fly Ash (kg/m3),Water (kg/m3),Superplasticizer (kg/m3),Coarse Aggregate (kg/m3),Fine Aggregate (kg/m3),Age (day),Compressive strength (MPa)
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64
1,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
2,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
4,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
5,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.3
6,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.03
7,380.0,95.0,0.0,228.0,0.0,932.0,594.0,365,43.7
8,380.0,95.0,0.0,228.0,0.0,932.0,594.0,28,36.45
9,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85


In [6]:
#splitting into 65% training, 10% test, 25% validation

x = Matrix(df[:, 1:8])
y = Matrix(df[:, 9:9])

xtrain = x[1:650, :]
ytrain = y[1:650]

xtest = x[651:750, :]
ytest = y[651:750]

xval = x[751:1030, :]
yval = y[751:1030]

280-element Vector{Float64}:
 44.09
 52.61
 59.76
 67.31
 69.66
 71.62
 74.17
 18.13
 22.53
 27.34
  ⋮
 31.88
 41.54
 39.46
 37.92
 44.28
 31.18
 23.7
 32.77
 32.4

In [19]:
xtrain

650×8 Matrix{Float64}:
 540.0    0.0  0.0  162.0  2.5  1040.0  676.0   28.0
 540.0    0.0  0.0  162.0  2.5  1055.0  676.0   28.0
 332.5  142.5  0.0  228.0  0.0   932.0  594.0  270.0
 332.5  142.5  0.0  228.0  0.0   932.0  594.0  365.0
 198.6  132.4  0.0  192.0  0.0   978.4  825.5  360.0
 266.0  114.0  0.0  228.0  0.0   932.0  670.0   90.0
 380.0   95.0  0.0  228.0  0.0   932.0  594.0  365.0
 380.0   95.0  0.0  228.0  0.0   932.0  594.0   28.0
 266.0  114.0  0.0  228.0  0.0   932.0  670.0   28.0
 475.0    0.0  0.0  228.0  0.0   932.0  594.0   28.0
   ⋮                               ⋮           
 250.0    0.0  0.0  182.0  0.0  1100.0  820.0    7.0
 250.0    0.0  0.0  182.0  0.0  1100.0  820.0   28.0
 350.0    0.0  0.0  186.0  0.0  1050.0  770.0    7.0
 350.0    0.0  0.0  186.0  0.0  1050.0  770.0   28.0
 203.5  305.3  0.0  203.5  0.0   963.4  630.0    7.0
 250.2  166.8  0.0  203.5  0.0   977.6  694.1   90.0
 157.0  236.0  0.0  192.0  0.0   935.4  781.2   90.0
 141.3  212.0  0.0  203.5  0

In [7]:
function dense_layer(x, p) 
	w, b = p 
	y = w*x .+ b
end

dense_layer (generic function with 1 method)

In [8]:
function train!(modelf, errf, p, x, labels, η, nsteps) 
	sample= rand(1:size(Matrix(x),2), 1000)
	for i= 1:nsteps 
		e(p) = errf(modelf(Matrix(x)[:,sample], p)[:], labels[sample])  
		p .-= η .* e'(p)
	end 
	return p
end

train! (generic function with 1 method)

In [10]:
function logit(x)
	1/(1+exp(-x))
end

logit (generic function with 1 method)

In [9]:
function relu(x) 
	return max(0,x)
end

relu (generic function with 1 method)

In [11]:
function logistic_regression(x::Matrix{T}, p::Vector) where T<:AbstractFloat
	logit.(dense_layer(x, p))
end

logistic_regression (generic function with 1 method)

In [12]:
function twolayer_logistic(x, p::Vector) where T<:AbstractFloat
	w1, b1, w2, b2 = p

	o1= relu.(dense_layer(x, [w1, b1]))
	o2 = dense_layer(o1, [w2, b2]) 
	o3 = logit.(o2)
	return o2
end

twolayer_logistic (generic function with 1 method)

In [13]:
function mse(ŷ, y)
	sum((ŷ .- y).^2) / length(y)
end

mse (generic function with 1 method)

In [14]:
function binary_crossentropy(ŷ::Vector{T}, y::Vector{T}) where T<:AbstractFloat
	-mean(y .* log.(ŷ) .+ (1.0 .- y) .* log.(1.0 .- ŷ))
end

binary_crossentropy (generic function with 1 method)

In [21]:
begin
	inputsize = 1030
	hidensize=1000
	nlabels = 1
	
	w1 = rand(hidensize, inputsize) .- 0.5
	b1 = rand(hidensize) .- 0.5
	
	w2 = rand(nlabels, hidensize) .- 0.5
	b2 = rand(nlabels) .- 0.5

	p = [w1,b1,w2,b2] 

	η = 0.01
	nsteps = 10000
	
	p = train!(twolayer_logistic, mse, p, x, xtrain, η, nsteps)
	
end

4-element Vector{Array{Float64}}:
 [-1577.4219731452333 -1577.2649385678872 … -1511.217425516079 -1822.5281879443692; 0.08863313680555562 0.17105418421454188 … -0.15930957050739658 -0.10092084063415785; … ; -15619.79422591567 -15728.181744547019 … -14676.868236088734 -14012.831661842949; -66090.58871785273 -66546.821170233 … -62105.19297547241 -59302.832277142144]
 [-12.720323647733814, -0.17487307034432276, -1.9007117471114484e23, -50.09732139759205, -1.4935674876831357e18, -1.3579954569802815, -2.8118193564543353e22, -6.902106364090651e22, 0.05213899441038361, -121.43851585944427  …  -5.336484933100822e19, 0.3340524616503182, -14.103270390856219, -87.93125567271072, -0.6591239646480206, -110.44037983828956, 0.2647212496699691, 0.31639542173602997, -21.03799130667726, -93.33457604125331]
 [-10678.206113569111 -0.2920543103467359 … -1.8424019656557108e6 -1.3711665061822336e6]
 [376.1794999999986]